异常值的处理

In [39]:
import pandas as pd
import numpy as np
from IPython.core. interactiveshell import InteractiveShell#jupyter运行输出的模块
import scipy.stats as stats#统计模块
import scipy
import matplotlib.pyplot as plt  #画图模块
from datetime import datetime   #时间模块

#输出矢量图 渲染矢量图 是一个魔法函数(magic functions)内嵌绘图
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#画图方便
#矢量图
InteractiveShell.ast_node_interactivity = 'all'#显示每一个运行结果
#设置行不限制数量 pd.set_option('display.max_rows',None)
#设置列不限制数量 pd.set_option('display.max_columns',None)

In [40]:
data= pd.read_csv('C:\Users\ignorance\Desktop\Python\数据\000001.csv') 
data['Day']=pd.to_datetime(data['Day'],format = '%Y/%m/%d')
data.set_index('Day',inplace= True) 
data.sort_values(by=['Day'],axis=0,ascending=True)#axis=1??? True是升序

,Preclose,Open,Highest,Lowest,Close,Volume,Money
Day,,,,,,,
1990-12-19,,96.05,99.98,95.79,99.98,126000,4.940000e+05
1990-12-20,99.98,104.30,104.39,99.98,104.39,19700,8.400000e+04
1990-12-21,104.39,109.07,109.13,103.73,109.13,2800,1.600000e+04
1990-12-24,109.13,113.57,114.55,109.13,114.55,3200,3.100000e+04
1990-12-25,114.55,120.09,120.25,114.55,120.25,1500,6.000000e+03
...,...,...,...,...,...,...,...
2022-07-25,3269.97,3269.71,3273.18,3243.03,3250.39,27124574400,3.480000e+11
2022-07-26,3250.39,3254.19,3282.41,3246.04,3277.44,25946867600,3.340000e+11
2022-07-27,3277.44,3271.78,3282.57,3265.73,3275.76,24913148500,3.400000e+11


In [41]:
daily_data=data['1995-01':'2022-07'].copy()
daily_data['Close']=pd.to_numeric(daily_data['Close'])
daily_data['Preclose']=pd.to_numeric(daily_data['Preclose'])
daily_data['Raw_return']=daily_data['Close']/daily_data['Preclose']- 1
daily_data['Log_return']=np.log(daily_data['Close'])-np.log(daily_data['Preclose'])
daily_data

,Preclose,Open,Highest,Lowest,Close,Volume,Money,Raw_return,Log_return
Day,,,,,,,,,
1995-01-03,647.87,637.72,647.71,630.53,639.88,23451800,1.806930e+08,-0.012333,-0.012409
1995-01-04,639.88,641.90,655.51,638.86,653.81,42222000,3.069230e+08,0.021770,0.021536
1995-01-05,653.81,655.38,657.52,645.81,646.89,43012300,3.015330e+08,-0.010584,-0.010641
1995-01-06,646.89,642.75,643.89,636.33,640.76,48748200,3.537580e+08,-0.009476,-0.009521
1995-01-09,640.76,637.52,637.55,625.04,626.00,50985100,3.985190e+08,-0.023035,-0.023305
...,...,...,...,...,...,...,...,...,...
2022-07-25,3269.97,3269.71,3273.18,3243.03,3250.39,27124574400,3.480000e+11,-0.005988,-0.006006
2022-07-26,3250.39,3254.19,3282.41,3246.04,3277.44,25946867600,3.340000e+11,0.008322,0.008288
2022-07-27,3277.44,3271.78,3282.57,3265.73,3275.76,24913148500,3.400000e+11,-0.000513,-0.000513


In [42]:
Month_data=daily_data.resample('M')['Log_return'].sum().to_frame()#resample('M')把日频度变成月频度 Q季度  Y年度
Month_data['Raw_return']=np.exp(Month_data['Log_return'])- 1
Month_data

,Log_return,Raw_return
Day,,
1995-01-31,-0.141139,-0.131631
1995-02-28,-0.023979,-0.023694
1995-03-31,0.163651,0.177803
1995-04-30,-0.109315,-0.103552
1995-05-31,0.188901,0.207922
...,...,...
2022-03-31,-0.062604,-0.060685
2022-04-30,-0.065154,-0.063077
2022-05-31,0.044724,0.045739


In [43]:
Quarter_data =daily_data.resample('Q')['Log_return'].sum().to_frame()
Quarter_data['Raw_return'] =np.exp(Quarter_data['Log_return'])-1
Quarter_data

,Log_return,Raw_return
Day,,
1995-03-31,-0.001467,-0.001466
1995-06-30,-0.025583,-0.025258
1995-09-30,0.135980,0.145660
1995-12-31,-0.263130,-0.231358
1996-03-31,0.001979,0.001981
...,...,...
2021-09-30,-0.006434,-0.006413
2021-12-31,0.019870,0.020069
2022-03-31,-0.112592,-0.106484


In [44]:
Year_data =daily_data.resample('Y')['Log_return'].sum().to_frame()
Year_data['Raw_return'] =np.exp(Year_data['Log_return'])-1
Year_data

,Log_return,Raw_return
Day,,
1995-12-31,-0.154200,-0.142899
1996-12-31,0.501639,0.651425
1997-12-31,0.264019,0.302153
1998-12-31,-0.040505,-0.039695
1999-12-31,0.175423,0.191750
2000-12-31,0.416917,0.517277
2001-12-31,-0.230898,-0.206180
2002-12-31,-0.192575,-0.175167
2003-12-31,0.097735,0.102670


固定比例法

将上下1%的值重新设置，若大于99%分位数的数值，则将其设置为99%分位数值，若低于1%分位数的数值，则将其重新设置为1%的数值

In [45]:
Month_data['2000-01':'2022-07']['Raw_return'].max()
Month_data['2000-01':'2022-07']['Raw_return'].min()

0.27446422361846134

-0.24631394466775336

In [46]:
Month_data_fix = Month_data['2000-01':'2022-07'].copy()
Month_data_fix

,Log_return,Raw_return
Day,,
2000-01-31,0.116219,0.123242
2000-02-29,0.110638,0.116990
2000-03-31,0.048741,0.049948
2000-04-30,0.019855,0.020053
2000-05-31,0.031218,0.031710
...,...,...
2022-03-31,-0.062604,-0.060685
2022-04-30,-0.065154,-0.063077
2022-05-31,0.044724,0.045739


In [47]:
Month_data_fix[Month_data_fix['Raw_return']>Month_data_fix['Raw_return'].quantile(0.99)]=Month_data_fix['Raw_return'].quantile(0.99)
Month_data_fix[Month_data_fix['Raw_return']<Month_data_fix['Raw_return'].quantile(0.01)]=Month_data_fix['Raw_return'].quantile(0.01)
Month_data_fix

,Log_return,Raw_return
Day,,
2000-01-31,0.116219,0.123242
2000-02-29,0.110638,0.116990
2000-03-31,0.048741,0.049948
2000-04-30,0.019855,0.020053
2000-05-31,0.031218,0.031710
...,...,...
2022-03-31,-0.062604,-0.060685
2022-04-30,-0.065154,-0.063077
2022-05-31,0.044724,0.045739


选取范围不同，变化明显

In [48]:
Month_data['2000-01':'2022-07']['Raw_return'].describe().round(8)
Month_data_fix['2000-01':'2022-07']['Raw_return'].describe().round(8)

count    271.000000
mean       0.005850
std        0.072477
min       -0.246314
25%       -0.038865
50%        0.006341
75%        0.045303
max        0.274464
Name: Raw_return, dtype: float64

count    271.000000
mean       0.005686
std        0.070397
min       -0.207600
25%       -0.038865
50%        0.006341
75%        0.045303
max        0.191280
Name: Raw_return, dtype: float64

均值标准差法

通常把3倍标准差之外的值都视为异常值，不过要注意的是样本均值和样本标准差都不是稳健统计量，其计算本身受极值的影响就非常大，所以可能会出现一种情况，那就是我们从数据分布图上能非常明显地看到异常点，但按照上面的计算方法，这个异常点可能仍在均值3倍标准差的范围内。因此按照这种方法剔除掉异常值后,需要重新观察数据的分布情况，看是否仍然存在显著异常点,若存在则继续重复上述步骤寻找异常点.

In [49]:
Month_data_ms= Month_data['2000-01':'2022-07'].copy()

up=Month_data_ms['Raw_return'].mean()+3*Month_data_ms['Raw_return'].std()
down=Month_data_ms['Raw_return'].mean()-3*Month_data_ms['Raw_return'].std()
Month_data_ms[Month_data_ms['Raw_return']>=up]=up
Month_data_ms[Month_data_ms['Raw_return']<=down]=down
Month_data_ms

,Log_return,Raw_return
Day,,
2000-01-31,0.116219,0.123242
2000-02-29,0.110638,0.116990
2000-03-31,0.048741,0.049948
2000-04-30,0.019855,0.020053
2000-05-31,0.031218,0.031710
...,...,...
2022-03-31,-0.062604,-0.060685
2022-04-30,-0.065154,-0.063077
2022-05-31,0.044724,0.045739


选取范围不同，变化不是很明显

In [50]:
Month_data['2000-01':'2022-07']['Raw_return'].describe().round(8)
Month_data_ms['2000-01':'2022-07']['Raw_return'].describe().round(8)

count    271.000000
mean       0.005850
std        0.072477
min       -0.246314
25%       -0.038865
50%        0.006341
75%        0.045303
max        0.274464
Name: Raw_return, dtype: float64

count    271.000000
mean       0.005869
std        0.071168
min       -0.211581
25%       -0.038865
50%        0.006341
75%        0.045303
max        0.223282
Name: Raw_return, dtype: float64

MAD法

MAD法是针对均值标准差方法的改进.,把均值和标准差替换成稳健统计量，样本均值用样本中位数代替，样本标准差用样本MAD( median absolute deviation)代替:

md = median (x,i= 1,2,… , n)

MAD = mean(|xi - md|,i = 1,2,…,n)

一般将偏离中位数3倍以上的数据作为异常值，和均值标淮差法相比，其中位数和MAD不受异常值的影响。

In [53]:
Month_data_median=Month_data['2000-01':'2022-07'].copy()
median=np.median(Month_data_median['Raw_return'])
MAD=np.mean(abs(Month_data_median['Raw_return']-median))

Month_data_median[Month_data_median['Raw_return']>3*MAD+median]=median+3 * MAD
Month_data_median[Month_data_median['Raw_return']<-3*MAD+median]=median-3 * MAD
Month_data_median

,Log_return,Raw_return
Day,,
2000-01-31,0.116219,0.123242
2000-02-29,0.110638,0.116990
2000-03-31,0.048741,0.049948
2000-04-30,0.019855,0.020053
2000-05-31,0.031218,0.031710
...,...,...
2022-03-31,-0.062604,-0.060685
2022-04-30,-0.065154,-0.063077
2022-05-31,0.044724,0.045739


In [54]:
Month_data['2000-01':'2022-07']['Raw_return'].describe().round(8)
Month_data_median['2000-01':'2022-07']['Raw_return'].describe().round(8)

count    271.000000
mean       0.005850
std        0.072477
min       -0.246314
25%       -0.038865
50%        0.006341
75%        0.045303
max        0.274464
Name: Raw_return, dtype: float64

count    271.000000
mean       0.006435
std        0.066373
min       -0.152583
25%       -0.038865
50%        0.006341
75%        0.045303
max        0.165266
Name: Raw_return, dtype: float64